In [ ]:

from importlib import reload
from simulation.simulation_builder import graph_builder as gb
from simulation.architectures.mnist_architectures import cnn_mnist_architecture
from tensorflow.python.client import device_lib
import sys
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
reload(gb)
g = gb.GraphBuilder(cnn_mnist_architecture,
                   learning_rate=0.01,
                   noise_list=[0.9, 0.8, 0.7, 0.6],
                   noise_type='dropout',
                   name='test')
#device_lib.list_local_devices()

def stdwrite(buff):
    sys.stdout.write('\r' + buff)
    sys.stdout.flush()

In [ ]:
batch_size = 50
with g._graph.as_default():
    from tensorflow.examples.tutorials.mnist import input_data
    data_path = 'simulation/data/mnist/'
    mnist = input_data.read_data_sets(data_path)
    test_feed_dict = g.create_feed_dict(mnist.test.images,
                                       mnist.test.labels,
                                       test=True)
    step = 0
    n_iters = n_iters = mnist.train.num_examples // batch_size
    with tf.Session() as sess:
        sess.run(g.variable_initializer)
        for epoch in range(10):
            for it in range(n_iters):
                step += 1
                X_batch, y_batch = mnist.train.next_batch(batch_size)

                evaluated = sess.run(g.get_train_ops(),
                                    feed_dict=g.create_feed_dict(X_batch,
                                                                y_batch))
                if it % 200 == 0:
                    evaluated = sess.run(g.get_train_ops(test=True),
                                        feed_dict=test_feed_dict)
                    loss = g.extract_evaluated_tensors(evaluated, 'loss')
                    buff = ''
                    for l in loss:
                        buff = buff + ' ,' + str(l)
                    stdwrite(buff)
                    #g.update_noise_vals(evaluated)
    

In [ ]:
import tensorflow as tf
from simulation.simulation_builder import graph_builder as gb

from simulation.architectures.mnist_architectures import nn_mnist_architecture
reload(gb)
print(1)
g = gb.GraphBuilder(nn_mnist_architecture,
                   learning_rate=0.01,
                   noise_list=[0.001, 0.003, 0.004, 0.009],
                   noise_type='random_normal',
                   name='test')

In [ ]:
batch_size = 50
with g._graph.as_default():
    from tensorflow.examples.tutorials.mnist import input_data
    data_path = 'simulation/data/mnist/'
    mnist = input_data.read_data_sets(data_path)
    test_feed_dict = g.create_feed_dict(mnist.test.images,
                                       mnist.test.labels,
                                       test=True)
    step = 0
    n_iters = n_iters = mnist.train.num_examples // batch_size
    with tf.Session() as sess:
        sess.run(g.variable_initializer)
        for epoch in range(10):
            for it in range(n_iters):
                step += 1
                X_batch, y_batch = mnist.train.next_batch(batch_size)

                evaluated = sess.run(g.get_train_ops(),
                                    feed_dict=g.create_feed_dict(X_batch,
                                                                y_batch))
                if it % 200 == 0:
                    evaluated = sess.run(g.get_train_ops(test=True),
                                        feed_dict=test_feed_dict)
                    loss = g.extract_evaluated_tensors(evaluated, 'loss')
                    buff = ''
                    for l in loss:
                        buff = buff + ' ,' + str(l)
                    stdwrite(buff)
                    g.update_noise_vals(evaluated)
    

In [ ]:
with graph.as_default():
    with tf.name_scope('Loss'):
        with tf.device('/gpu:0'):
            cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                            logits=logits)
        with tf.device('/cpu:0'):
            loss = tf.reduce_mean(cross_entropy, name='loss')
    with tf.name_scope('Accuracy'):
        with tf.device('/cpu:0'):
            y_pred = tf.nn.in_top_k(predictions=logits, targets=y, k=1)
            accuracy = tf.reduce_mean(tf.cast(x=y_pred, dtype=tf.float32),
                                     name='accuracy')
    with tf.name_scope('Optimizer'):
        with tf.device('/gpu:0'):
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
            train_op = optimizer.minimize(loss)

    

In [ ]:
with graph.as_default():
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        print(train_op.device)


In [ ]:
from simulation.simulation_builder import optimizers as o
opt = o.GDOptimizer(0.01, 0, [0.01, 0.02])
        

In [ ]:
import tensorflow as tf
